# Patch loader experimentation notebook

The patch loader is going to be our harness **entry point**. Where we are going to **load**, **apply**, and **manage** LLM-generated patches to repositories.
First of all, what is a patch?

A patch is a textual representation of code changes, basically a diff showing what lines were added, removed, or modified in a repository.
When an LLM-based agent tries to solve a **SWE-bench task**, it receives , for example, something like:

“Here is a failing test and the project context. Please modify the code so the test passes."

The agent reads the given code and generates a *patch* doing a *diff* operation between the old and new code strings
```text
diff --git a/app/utils.py b/app/utils.py
index 3a4bcd2..e8f7aab 100644
--- a/app/utils.py
+++ b/app/utils.py
@@ -42,7 +42,10 @@ def get_value(key):
     value = cache.get(key)
-    if value:               # "-" indicates the old code
-        return value
-    return None
+    if value is not None:   # "+" indicates the generated code from the agent to solve the issue"
+        return value
+    else:
+        logger.warning(f"Cache miss for {key}")
+        return None
´´´



**SWE-bench** and its variations already provide generated patches from many models such as GPT,Claude,Gemini... We will be using those in order to test that our pipeline works correctly

So the patch loader is in charge of taking that string and:

* Loads the base repo (same commit SWE-bench uses).

* Applies the patch via git apply.

* Returns the patched repo so the rest of your modules can:

    * run static checks (flake8, mypy),

    * run dynamic verification (pytest, fuzzing, coverage),

    * and finally see if the issue is fixed